In [100]:
import pandas as pd
import numpy as np
import seaborn as sns

df = pd.read_csv("/Users/ipatel/Documents/Amit/Python/Practice/p1/notbook/raw.csv")
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1, inplace=True)
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [101]:
df.shape

(891, 8)

In [102]:
df.isnull().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [103]:
df['Family'] = df['SibSp'] + df['Parch']

In [104]:

X = df.drop(columns=['Survived', 'SibSp', 'Parch'], axis=1)
y = df['Survived']

y.value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [105]:
X.sample(5)

,Pclass,Sex,Age,Fare,Embarked,Family
235,3,female,NaN,7.5500,S,0
136,1,female,19.0,26.2833,S,2
50,3,male,7.0,39.6875,S,5
178,2,male,30.0,13.0000,S,0
447,1,male,34.0,26.5500,S,0


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split
import numpy as np

from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report


# Assuming X and y are already defined
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

num_columns = X_train.select_dtypes(include=np.number).columns.tolist()
cat_columns = X_train.select_dtypes(exclude=np.number).columns.tolist()
print(f'cat_columns {cat_columns}')
print(f'num_columns {num_columns}')

# Define Transformers Numerical and Categorical Columns
imput_numeric_col = SimpleImputer(strategy="median")
num_scaler = StandardScaler()
imput_category_col = SimpleImputer(strategy="most_frequent")
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Combined Column Transformer for categorical and numerical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num_col', Pipeline(steps=[
            ('impute', imput_numeric_col),
            ('scale', num_scaler)
        ]), num_columns),
        ('cat_col', Pipeline(steps=[
            ('impute', imput_category_col),
            ('onehot', ohe)
        ]), cat_columns)
    ]
)

model_pipeline = ImbPipeline([
    ('preprocess', preprocessor),
    ('smote', SMOTE(random_state=42)),
    ('classifier', LogisticRegression())
])

# Train model
model_pipeline.fit(X_train, y_train)

# Evaluate model
score = model_pipeline.score(X_test, y_test)
print(f'Model Accuracy: {score:.4f}')

y_pred_smote = model_pipeline.predict(X_test)
print("\nResults with SMOTE:")
print(classification_report(y_test, y_pred_smote))

cat_columns ['Sex', 'Embarked']
num_columns ['Pclass', 'Age', 'Fare', 'Family']
Model Accuracy: 0.8156

Results with SMOTE:
              precision    recall  f1-score   support

           0       0.87      0.81      0.84       105
           1       0.75      0.82      0.79        74

    accuracy                           0.82       179
   macro avg       0.81      0.82      0.81       179
weighted avg       0.82      0.82      0.82       179



{'0': {'precision': 0.8673469387755102,
  'recall': 0.8095238095238095,
  'f1-score': 0.8374384236453202,
  'support': 105.0},
 '1': {'precision': 0.7530864197530864,
  'recall': 0.8243243243243243,
  'f1-score': 0.7870967741935484,
  'support': 74.0},
 'accuracy': 0.8156424581005587,
 'macro avg': {'precision': 0.8102166792642983,
  'recall': 0.8169240669240669,
  'f1-score': 0.8122675989194343,
  'support': 179.0},
 'weighted avg': {'precision': 0.8201107465539496,
  'recall': 0.8156424581005587,
  'f1-score': 0.8166267920283865,
  'support': 179.0}}